## Importing packages and csvs

In [1]:
# importing packages
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
#from ET_mysql_key import ET_key
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# setting up csv variables
WSJ_salaries_college_file = "../Data_Files/college_salaries/salaries-by-college-type.csv" # has median salary data 
CC_institution_details_file = "../Data_Files/college_completion/cc_institution_details.csv" # has all data incl. school name
CC_institution_grads_file = "../Data_Files/college_completion/cc_institution_grads.csv" # has gender data
# WSJ_salaries_region_file = "../Data_Files/college_salaries/salaries-by-region.csv"
# CC_state_sector_details_file = "../Data_Files/college_completion/cc_state_sector_details.csv"
# CC_state_sector_grads_file = "../Data_Files/college_completion/cc_state_sector_grads.csv"

# testing 
# PEPS_closed_schools_file = "/Data_Files/closedschoolsearch.csv"

## Extraction

In [3]:
# extracting csvs into dataframes
WSJ_college_raw_df = pd.read_csv(WSJ_salaries_college_file)
# WSJ_college_raw_df.head()

In [4]:
CC_institution_details_raw_df = pd.read_csv(CC_institution_details_file)
# CC_institution_details_raw_df.head(2)
# CC_institution_details_raw_df.shape

In [5]:
CC_institution_grads_raw_df = pd.read_csv(CC_institution_grads_file)
# CC_institution_grads_raw_df.head()

## Cleaning

In [6]:
# remove columns as needed and set as the real df
WSJ_college_cols = ["School Name", "School Type", "Starting Median Salary", "Mid-Career Median Salary"]
WSJ_college_df = WSJ_college_raw_df[WSJ_college_cols].copy()
WSJ_college_df = WSJ_college_df.rename(columns={"School Name": "school_name", "School Type": "school_type", 
                                                "Starting Median Salary": "median_salary_start", 
                                                "Mid-Career Median Salary": "median_salary_mid"})
WSJ_college_Df = WSJ_college_df.dropna(how="any")
WSJ_college_Df.head()

school_name   school_type median_salary_start  \
0               Amherst College  Liberal Arts         $54,500.00    
1  Appalachian State University         State         $40,400.00    
2      Arizona State University         Party         $47,400.00    
3      Arizona State University         State         $47,400.00    
4     Arkansas State University         State         $38,700.00    

  median_salary_mid  
0      $107,000.00   
1       $69,100.00   
2       $84,100.00   
3       $84,100.00   
4       $63,300.00

In [7]:
# remove columns as needed and set as the real df
CC_institution_details_cols = ["unitid", "chronname", "state", "level","control","basic",
                               "student_count","aid_value","grad_100_value","grad_100_percentile",
                              "grad_150_value","grad_150_percentile"]
CC_institution_details_df = CC_institution_details_raw_df[CC_institution_details_cols].drop_duplicates().copy()
CC_institution_details_df = CC_institution_details_df.rename(columns={"unitid": "school_id", 
                                                                      "chronname":"school_name"})
CC_institution_details_Df = CC_institution_details_df.dropna(how="any")
CC_institution_details_Df.head()

school_id                               school_name       state   level  \
0     222178              Abilene Christian University       Texas  4-year   
2     172866                           Academy College   Minnesota  4-year   
3     108232                 Academy of Art University  California  4-year   
4     441201  Academy of Court Reporting at Cincinnati        Ohio  2-year   
5     200633   Academy of Court Reporting at Cleveland        Ohio  2-year   

                  control                                              basic  \
0  Private not-for-profit  Masters Colleges and Universities--larger prog...   
2      Private for-profit              Baccalaureate and Associates Colleges   
3      Private for-profit                  Schools of art- music- and design   
4      Private for-profit                     Associates--Private For-profit   
5      Private for-profit                     Associates--Private For-profit   

   student_count  aid_value  grad_100_value  grad_100_percentile  \
0           3806     9405.0            37.0                 44.0   
2            236     4564.0           100.0                 97.0   
3          12181     5342.0             5.6                 36.0   
4            149     4898.0             0.0                  0.0   
5            399     5544.0            35.7                 41.0   

   grad_150_value  grad_150_percentile  
0            56.9                 53.0  
2           100.0                 96.0  
3            29.2                 58.0  
4             0.0                  0.0  
5            35.7                 15.0

In [8]:
# removing year column and dropping duplicates
CC_institution_grads_cols = ["unitid", "gender", "race", "cohort"]
CC_institution_grads_df = CC_institution_grads_raw_df[CC_institution_grads_cols].drop_duplicates().copy()
CC_institution_grads_df = CC_institution_grads_df.rename(columns={"unitid":"school_id"})
CC_institution_grads_Df = CC_institution_grads_df.dropna(how="any")
CC_institution_grads_Df.head()

school_id gender race  cohort
0     101462      B    X  2y all
1     101471      B    X  2y all
2     101499      B    X  2y all
3     101505      B    X  2y all
4     101514      B    X  2y all

## Loading

In [9]:
# creating database connection
connection_string = "root:pass@localhost/Colleges"
engine = create_engine(f'mysql://{connection_string}')

In [10]:
# confirming tables 
engine.table_names()


['Demographics', 'Grad_Rates', 'Salaries']

In [12]:
# loading dataframes into database
WSJ_college_Df.to_sql(name='Salaries', con=engine, if_exists='replace', index=True)

CC_institution_details_Df.to_sql(name='Grad_Rates', con=engine, if_exists='replace', index=True)

CC_institution_grads_Df.to_sql(name='Demographics', con=engine, if_exists='replace', index=True)






In [14]:
engine.table_names()

['Demographics', 'Grad_Rates', 'Salaries']

In [16]:
pd.read_sql("select * from Demographics",con = engine)

index  school_id gender race    cohort
0            0     101462      B    X    2y all
1            1     101471      B    X    2y all
2            2     101499      B    X    2y all
3            3     101505      B    X    2y all
4            4     101514      B    X    2y all
5            5     101648      B    X    2y all
6            6     101897      B    X    2y all
7            7     101949      B    X    2y all
8            8     103927      B    X    2y all
9            9     104425      B    X    2y all
10          10     104708      B    X    2y all
11          11     105206      B    X    2y all
12          12     105349      B    X    2y all
13          13     105747      B    X    2y all
14          14     106148      B    X    2y all
15          15     106795      B    X    2y all
16          16     107327      B    X    2y all
17          17     107585      B    X    2y all
18          18     107619      B    X    2y all
19          19     107725      B    X    2y all
20          20     107974      B    X    2y all
21          21     108667      B    X    2y all
22          22     108807      B    X    2y all
23          23     109208      B    X    2y all
24          24     111461      B    X    2y all
25          25     111638      B    X    2y all
26          26     111887      B    X    2y all
27          27     111896      B    X    2y all
28          28     111939      B    X    2y all
29          29     112385      B    X    2y all
...        ...        ...    ...  ...       ...
107826  953098     164492      F   Ai  4y other
107827  953099     164632      F   Ai  4y other
107828  953100     164641      F   Ai  4y other
107829  953101     164924      F   Ai  4y other
107830  953102     164933      F   Ai  4y other
107831  953103     165024      F   Ai  4y other
107832  953104     165574      F   Ai  4y other
107833  953105     165671      F   Ai  4y other
107834  953106     165820      F   Ai  4y other
107835  953107     165866      F   Ai  4y other
107836  953108     166513      F   Ai  4y other
107837  953109     166656      F   Ai  4y other
107838  953110     166911      F   Ai  4y other
107839  953111     166939      F   Ai  4y other
107840  953112     166948      F   Ai  4y other
107841  953113     167057      F   Ai  4y other
107842  953114     167251      F   Ai  4y other
107843  953115     167260      F   Ai  4y other
107844  953116     167598      F   Ai  4y other
107845  953117     167729      F   Ai  4y other
107846  953118     167792      F   Ai  4y other
107847  953119     167835      F   Ai  4y other
107848  953120     167987      F   Ai  4y other
107849  953121     168342      F   Ai  4y other
107850  953122     168528      F   Ai  4y other
107851  953123     168591      F   Ai  4y other
107852  953124     168740      F   Ai  4y other
107853  953125     169716      F   Ai  4y other
107854  953126     170082      F   Ai  4y other
107855  953127     170091      F   Ai  4y other

[107856 rows x 5 columns]